In [1]:
import cv2
import torch
from PIL import Image
import os

In [2]:
from tqdm.notebook import tqdm

In [3]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [4]:
path = '../Satellite-Aerial-Image-Retrieval/output/'

images_path = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.jpeg' in file:
            images_path.append(os.path.join(r, file))

In [5]:
model_name = "yolov5s_576_d15_t0.35_s0.3_bs3_wd0.0.05_lr0.001_newdat"

In [6]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=f'weights/{model_name}.pt') 

Using cache found in /home/hue/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-9-11 torch 1.9.0+cu102 CUDA:0 (NVIDIA GeForce MX250, 2002.625MB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
Adding AutoShape... 


In [7]:
model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.0  # NMS IoU threshold (0-1)

In [8]:
import time

In [9]:
bs = 50
for path in tqdm(images_path[:1]):
    name = os.path.basename(path)[:-5]
    output_path_raw = f"output/{model_name}/{name}"
    os.makedirs(output_path_raw, exist_ok=True)
    output_path_annotated = f"output/{model_name}/{name}_annotated"
    os.makedirs(output_path_annotated, exist_ok=True)
    bgr_image = cv2.imread(path)
    image_rgb = cv2.cvtColor(bgr_image,cv2.COLOR_BGR2RGB)  # OpenCV image (BGR to RGB)
    tiles = [image for _,_, image in sliding_window(image_rgb, 576, (800, 800)) if image.shape[0]==image.shape[1]]
    
#     detected_areas = []
    for i in tqdm(range(0, len(tiles), bs), leave=False):
        raw_chunk = tiles[i:i + bs][:]
        chunk = tiles[i:i + bs]
        results = model(chunk, size=576)
#         results.save()
        results.render()
        for idx, result in enumerate(results.pred):
            if len(result) != 0:
#                 detected_areas.append([f"{name[:-5]}_{i}_{idx}.png",raw_chunk[idx]])
                cv2.imwrite(f"{output_path_raw}/{name}_{i}_{idx}.png",  cv2.cvtColor(raw_chunk[idx], cv2.COLOR_RGB2BGR))
                cv2.imwrite(f"{output_path_annotated}/{name}_{i}_{idx}.png",  cv2.cvtColor(chunk[idx], cv2.COLOR_RGB2BGR))
#     tqdm.write(f"Finding Caravansara in image '{name}' Finished, Sleeping for 1 Min")
#     time.sleep(60)

In [10]:
results.pred

[tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([], device='cuda:0', size=(0, 6)),
 tensor([],

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.figure(figsize=(10, 10))
# plt.imshow()
# plt.show()